In [1]:
from dev import *


record_keys_per_dataset_source = {}
print("#" * 60)
print("Checking for inconsistency in record types")
for dataset_name in DATASET_NAMES.keys():
    for source_name in WATERMARK_METHODS.keys():
        record_keys_per_result_type = {}
        result_types_expected = (
            ["status"]
            + (["reverse"] if source_name in ["tree_ring"] else [])
            + ["decode", "metric"]
        )
        for result_type in result_types_expected:
            record_keys_per_result_type[result_type] = set(
                [
                    (key[1], key[2])
                    for key in get_all_json_paths(
                        lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
                            (_dataset_name == dataset_name)
                            and (
                                _source_name == source_name
                                if source_name != "real"
                                else _source_name.startswith("real")
                            )
                            and (_result_type == result_type)
                        )
                    ).keys()
                ]
            )
        for result_type in result_types_expected:
            for attack_name, attack_strength in (
                set.union(*record_keys_per_result_type.values())
                - record_keys_per_result_type[result_type]
            ):
                print(
                    f"Missing {dataset_name}, {source_name}, {attack_name}, {attack_strength}, {result_type}"
                )
        record_keys_per_dataset_source[(dataset_name, source_name)] = set.intersection(
            *record_keys_per_result_type.values()
        )


print("#" * 60)
print("Checking for inconsistency across datasetes and sources")
for dataset_name in DATASET_NAMES.keys():
    for source_name in WATERMARK_METHODS.keys():
        for attack_name, attack_strength in (
            set.union(*record_keys_per_dataset_source.values())
            - record_keys_per_dataset_source[(dataset_name, source_name)]
        ):
            print(
                f"Missing {dataset_name}, {source_name}, {attack_name}, {attack_strength}"
            )

############################################################
Checking for inconsistency in record types
Missing diffusiondb, tree_ring, 2x_regen, 10.0, reverse
Missing diffusiondb, tree_ring, adv_emb_clip_untg_alphaRatio_0.05_step_200, 2.0, decode
Missing diffusiondb, tree_ring, 2x_regen, 10.0, decode
Missing diffusiondb, tree_ring, 2x_regen, 10.0, metric
Missing diffusiondb, stable_sig, 2x_regen, 10.0, metric
Missing diffusiondb, stegastamp, 2x_regen, 10.0, decode
Missing diffusiondb, stegastamp, 2x_regen, 10.0, metric
Missing mscoco, tree_ring, 2x_regen, 10.0, status
Missing dalle3, tree_ring, 2x_regen, 10.0, status
############################################################
Checking for inconsistency across datasetes and sources
Missing diffusiondb, tree_ring, adv_emb_clip_untg_alphaRatio_0.05_step_200, 2.0
Missing diffusiondb, tree_ring, 2x_regen, 10.0
Missing diffusiondb, stable_sig, 2x_regen, 10.0
Missing diffusiondb, stegastamp, 2x_regen, 10.0
Missing mscoco, tree_ring, 2x_rege